In [44]:
import numpy as np
from pymatreader import read_mat
import os
from tensorflow.python.keras import Input, Model, Sequential
import nengo
import tensorflow as tf
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, \
    Conv3D, LSTM
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K

In [45]:
channels_of_interest = ['O9', 'CP1', 'CP2', 'O10', 'P7', 'P3', 'Pz',
            'P4', 'P8', 'PO7', 'PO3', 'Oz', 'PO4', 'PO8']

def get_participant_data(file):
    mat = read_mat(file)
    bci = mat['bciexp']
    data = bci['data']
    labels = bci['label']

    channel_indexes = np.array([labels.index(ch) for ch in channels_of_interest])

    return data, bci['intention']

def get_dataset(dataset_files):
    X, Y = [], []
    for file in dataset_files:
        features, labels = get_participant_data(file)
        features = features.reshape(-1, 30, 4000)
        X.append(features)
        Y.append(labels)

    X = np.concatenate(X)
    Y = np.concatenate(Y)
    return X, Y


In [46]:
seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)

dataset_path = 'dataset'
dataset_files = []
for file in os.listdir(dataset_path):
    dataset_files.append(os.path.join(dataset_path, file))

features, labels = get_dataset(dataset_files)
features = features.reshape(features.shape[0], 30, 40, -1)

labels = labels.reshape((-1, 1))
labels = OneHotEncoder().fit_transform(labels).toarray()

In [47]:
print(features.shape)
print(labels.shape)

(2976, 30, 40, 100)
(2976, 2)


In [48]:
def cnn_model():
    inp = Input(shape=features.shape[1:], name='input_layer')
    conv1 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
    dropout1 = Dropout(0.2, seed=seed)(conv1)
    avg_pool1 = AveragePooling2D(pool_size=(2, 2))(dropout1)
    conv2 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(avg_pool1)
    dropout2 = Dropout(0.2)(conv2)
    avg_pool2 = AveragePooling2D(pool_size=(2, 2))(dropout2)
    flatten = Flatten()(avg_pool2)
    dense1 = Dense(512, activation='relu')(flatten)
    batch_norm = BatchNormalization()(dense1)
    dense2 = Dense(256, activation='relu')(batch_norm)
    output = Dense(2, activation='sigmoid', name='output_layer')(dense2)

    return Model(inputs=inp, outputs=output)

In [49]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=.25, random_state=seed, shuffle=True)

model = cnn_model()
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )
model.fit(x_train, y_train, epochs=65)
eval = model.evaluate(x_test, y_test)
print('Accuracy: ', eval[1] * 100)

Epoch 1/65
70/70 [==============================] - 7s 83ms/step - loss: 0.7643 - accuracy: 0.5160